# CloudHealth API Requests for Cloud Optimization Reviews

This repo contains multiple CloudHealth request for common optimization analyses. Input your CloudHealth API token in the quotes associated with my_api_token and then input the customer ID inside the quotes associated with client_id to create strings. Make sure to run cells in order to load the variables and libraries for the API requests to utilize.

In [16]:
import requests
import json 
import datetime
import pandas as pd

In [8]:
my_api_token = ""
my_headers = {'Authorization': 'Bearer ' + my_api_token}

In [9]:
client_id = ""

In [11]:
#Volume Savings via GP2 to GP3 Upgrade 

In [12]:
response = requests.get("https://chapi.cloudhealthtech.com/api/search?&api_version=2&client_api_id=" + client_id + "&name=AwsVolume&query=volume_type='gp2'+and+is_active=1&fields=volume_type,size,name,price_per_month,in_use,account.name", headers=my_headers)

#Converts JSON to a list of dictionaries with the information of every active volume 
volumes = json.loads(response.text)

#Obtain customer name 
customer = volumes[0]['account']['name'].split('-')[0].strip()

#Initialize gp2 count and cost
attached_gp2_volumes = 0
cost_of_volumes = 0

#Only count if volume is attached and sum monthly costs of attached volumes 
for volume in volumes:
    if volume["in_use"]:
        attached_gp2_volumes += 1
        cost_of_volumes += float(volume["price_per_month"][1:])

#Convert volume cost to savings (20% for gp2 to gp3 converion), print results 
savings = round(0.2 * cost_of_volumes, 2)

#Find largest volume 
sizeList = [volume['size'] for volume in volumes]
largest = max(sizeList)

#Print analysis
print(f"Customer: {customer}")
print(f'Number of attached gp2 volumes: {attached_gp2_volumes}')
print(f'Monthly savings by converting to gp3: ${savings}')
print(f'Largest Volume: {largest} GB')


Customer: Innocean Worldwide Americas
Number of attached gp2 volumes: 25
Monthly savings by converting to gp3: $61.08
Largest Volume: 250 GB


In [14]:
#Snapshot Summary

In [17]:
response = requests.get("https://chapi.cloudhealthtech.com/api/search?&api_version=2&client_api_id=" + client_id + "&name=AwsSnapshot&query=is_active=1&fields=name,size,create_date,account.name", headers=my_headers)

#Converts JSON to a list of dictionaries with the information of every active snapshots 
snapshots = json.loads(response.text)

#Obtain customer name 
customer = snapshots[0]['account']['name'].split('-')[0].strip()

#Initialize counter for number of snapshots created before 2021
before2021 = 0

#Create new key:value pair for the age of each snapshot based on today's date 
for snapshot in snapshots:
    snapshot['age'] = (datetime.date.today() - datetime.datetime.strptime(snapshot['create_date'], '%Y-%m-%dT%H:%M:%SZ').date()).days
    if (datetime.datetime.strptime(snapshot['create_date'], '%Y-%m-%dT%H:%M:%SZ').year < 2021):
        before2021 += 1
    
#Find total number of snapshots, average age of snapshots, and % created before 2021
totalSnapshots = len(snapshots)
averageAge = int(sum(snapshot['age'] for snapshot in snapshots) / totalSnapshots)
percentBefore2021 = int((before2021/totalSnapshots)*100)

#Find oldest snapshot 
ageList = [snapshot['age'] for snapshot in snapshots]
oldest = max(ageList)
oldestIndex = ageList.index(oldest)
oldestSnapshot = datetime.datetime.strptime(snapshots[oldestIndex]['create_date'], '%Y-%m-%dT%H:%M:%SZ').date()

#Find largest snapshot
sizeList = [snapshot['size'] for snapshot in snapshots]
largest = max(sizeList)

#Print analysis
print(f"Customer: {customer}")
print(f'{totalSnapshots} snapshots')
print(f'Average Age: {averageAge} days')
print(f'{percentBefore2021}% created before 2021 ({before2021} snapshots)')
print(f'Oldest Snapshot: {oldestSnapshot}')
print(f'Largest Snapshot: {largest} GB')

Customer: Innocean Worldwide Americas
120 snapshots
Average Age: 177 days
20% created before 2021 (24 snapshots)
Oldest Snapshot: 2017-12-13
Largest Snapshot: 100 GB


In [18]:
#Summary of Past Three Months Costs

In [19]:
response = requests.get("https://chapi.cloudhealthtech.com/olap_reports/cost/history?&api_version=2&client_api_id=" + client_id + "&dimensions[]=time&dimensions[]=AWS-Service-Category&measure[]=cost&interval=monthly&filters[]=time:select:-2,-3,-4", headers=my_headers)

#Converts JSON to a list of dictionaries, initialize nested list to hold service items and past three months of data 
service_items = json.loads(response.text)
data = [[], [], [], []]

#Get previous three month labels
lastMonth = date.today().replace(day=1) - timedelta(days=1)
twoMonths = lastMonth.replace(day=1) - timedelta(days=1)
threeMonths = twoMonths.replace(day=1) - timedelta(days=1)

#Initalize labels for data 
labels = ['Service Items', threeMonths.strftime("%m-%Y"), twoMonths.strftime("%m-%Y"), lastMonth.strftime("%m-%Y")]

#Fill in data with service items, note that this pulls Direct listings (i.e. EC2 direct is the sum of all associated EC2 charges), 
# could remove with parent: -1 with exceptions for services with no children (workspace)
iterate = 0
for item in service_items['dimensions'][1]['AWS-Service-Category']:
    data[0].append(item['label'])
    iterate += 1

#Fill in data with costs from past three months 
for i in range(10, 13):
    for item in service_items['data'][i]:
        data[i-9].append(item[0])

#Convert nested list data into a dictionary and then into a dataframe 
dictionary = dict(zip(labels, data))
df = pd.DataFrame(data=dictionary)

#Create column with % change from previous month and the month before
df['Monthly % Change'] = ((df[twoMonths.strftime("%m-%Y")] - df[lastMonth.strftime("%m-%Y")])/df[twoMonths.strftime("%m-%Y")])*100

#Create file name based on customer id and date, sort the dataframe by previous month's costs and export to csv
name = client_id + "_" + date.today().strftime('%Y-%m-%d') + "cost_history"
filename = "%s.csv" % name
sorted_df = df.sort_values(by=[lastMonth.strftime("%m-%Y")], ascending=False)
sorted_df.to_csv(filename, index=False)

In [22]:
#S3 Bucket Summary

In [23]:
response = requests.get("https://chapi.cloudhealthtech.com/olap_reports/cost/s3?&api_version=2&client_api_id=" + client_id + "&dimensions[]=time&dimensions[]=S3-Bucket&measure[]=s3_cost_storage&interval=monthly&filters[]=time:select:-1, -2, -3", headers=my_headers)

#Converts JSON to a list of dictionaries, initialize nested list to s3 buckets and their previous month's costs 
buckets = json.loads(response.text)
data = [[], [], [], []]

#Get previous three month labels
lastMonth = date.today().replace(day=1) - timedelta(days=1)
twoMonths = lastMonth.replace(day=1) - timedelta(days=1)
threeMonths = twoMonths.replace(day=1) - timedelta(days=1)

#Initalize labels for data 
labels = ['Service Items', threeMonths.strftime("%m-%Y"), twoMonths.strftime("%m-%Y"), lastMonth.strftime("%m-%Y")]

#Fill in data with service items, note that this pulls Direct listings (i.e. EC2 direct is the sum of all associated EC2 charges), 
# could remove with parent: -1 with exceptions for services with no children (workspace)
iterate = 0
for item in buckets['dimensions'][1]['S3-Bucket']:
    data[0].append(item['label'])
    iterate += 1

#Fill in data with costs from past three months 
for i in range(10, 13):
    for item in buckets['data'][i]:
        data[i-9].append(item[0])

#Convert nested list data into a dictionary and then into a dataframe 
dictionary = dict(zip(labels, data))
df = pd.DataFrame(data=dictionary)

#Create column with % change from previous month and the month before
df['Monthly % Change'] = ((df[twoMonths.strftime("%m-%Y")] - df[lastMonth.strftime("%m-%Y")])/df[twoMonths.strftime("%m-%Y")])*100

#Create file name based on customer id and date, sort the dataframe by previous month's costs and export to csv
name = client_id + "_" + date.today().strftime('%Y-%m-%d') + "_s3"
filename = "%s.csv" % name
sorted_df = df.sort_values(by=[lastMonth.strftime("%m-%Y")], ascending=False)
sorted_df.to_csv(filename, index=False)